# Classification Mesh 


Le MeSH (Medical Subject Headings) est un vocabulaire contrôlé utilisé pour organiser et indexer les articles médicaux. Il est structuré de manière hiérarchique en “arbres” (MeSH trees), où chaque concept est associé à un code unique appelé numéro d’arbre.

“Catégorie principale des Maladies nutritionnelles et métaboliques” est ainsi associé au code C14, où “C” représente la catégorie des maladies et “14” spécifie le groupe des maladies nutritionnelles et métaboliques. Cette organisation facilite la navigation du général au spécifique, aidant ainsi les chercheurs à trouver des informations précises.

Mais il est aussi d'être plus precis en selectionnant des maladies plus loin dans l'**arborescences**.

Par le Diabete de type 1 correspond au code MeSH C18.452.394.750.124 qui se décompose comme suit :

*   C : Catégorie des maladies
*   C18 : Catégorie principale des Maladies nutritionnelles et métaboliques.
*   C18.452 : Sous-catégorie des Maladies métaboliques.
*   C18.452.394 : Sous-sous-catégorie des Troubles du métabolisme du glucose.
*   C18.452.394.750 : Sous-sous-sous-catégorie du Diabète sucré.
*   C18.452.394.750.124 : Terme spécifique pour le Diabète sucré de type 1.

Cette hiérarchie reflète l’organisation des termes MeSH, permettant de situer chaque condition médicale dans un contexte plus large et de naviguer efficacement dans les informations médicales.

In [1]:
import pandas as pd
import numpy as np

In [7]:
import sys
import os

# Ajouter le dossier courant au PATH si ce n'est pas déjà fait
sys.path.append(os.path.abspath("."))

import MeSH
print(MeSH.Hello())

Hello from MeSH!


Le dictionnaire est disponible au format XML à cet [URL](https://nlmpubs.nlm.nih.gov/projects/mesh/MESH_FILES/xmlmesh/?_gl=1*1vhz8wv*_ga*MTkwNTMyNjEyOC4xNzM2Njg2Mzcy*_ga_7147EPK006*MTczNzEwNTQ1MS4xMi4xLjE3MzcxMDU4OTMuMC4wLjA.*_ga_P1FPTH9PL4*MTczNzEwNTQ1MS4xNC4xLjE3MzcxMDU4OTMuMC4wLjA)

In [3]:
import xml.etree.ElementTree as ET
import requests
import os

# URL et chemin local du fichier XML
url_fichier = "https://nlmpubs.nlm.nih.gov/projects/mesh/MESH_FILES/xmlmesh/desc2025.xml"
chemin_fichier = "data/desc2025.xml"  # Nom du fichier local

# Vérifier si le fichier existe déjà
if not os.path.exists(chemin_fichier):
    try:
        response = requests.get(url_fichier)
        response.raise_for_status()  # Vérifie si la requête a réussi

        with open(chemin_fichier, "wb") as fichier:
            fichier.write(response.content)

        print(f"Fichier téléchargé avec succès : {chemin_fichier}")
    except requests.RequestException as e:
        print(f"Erreur lors du téléchargement du fichier XML : {e}")
        exit(1)  # Arrête l'exécution en cas d'échec
else:
    print(f"Le fichier {chemin_fichier} existe déjà. Pas de téléchargement nécessaire.")

# Fonction pour analyser un fichier XML
def analyser_xml(chemin):
    try:
        arbre = ET.parse(chemin)
        racine = arbre.getroot()
        print(f"Fichier XML analysé avec succès : {chemin}")
        return racine
    except ET.ParseError as e:
        print(f"Erreur lors de l'analyse du fichier XML : {e}")
    except FileNotFoundError:
        print(f"Le fichier spécifié est introuvable : {chemin}")
    except Exception as e:
        print(f"Une erreur inattendue s'est produite : {e}")

# Analyse du fichier téléchargé
racine = analyser_xml(chemin_fichier)

Le fichier data/desc2025.xml existe déjà. Pas de téléchargement nécessaire.
Fichier XML analysé avec succès : data/desc2025.xml


## Chargement données

In [4]:
df = pd.read_parquet('data/df_final')
df.head(3)

,pmcid,title,gender,case_text,keywords,major_mesh_terms,mesh_terms,journal,doi
0,PMC5287946,Malignant adenohypophysis spindle cell oncocyt...,Male,A 30-year-old man came to Peking Union Medical...,None,"[Adenoma, Oxyphilic / pathology, Ki-67 Antigen...","[Adenoma, Oxyphilic / pathology, Ki-67 Antigen...",Medicine (Baltimore),10.1097/MD.0000000000005657
1,PMC3678730,Misclassification of pregnancy-related deaths ...,Female,"For each death, the DSS has systematically rec...",None,"[Cause of Death, Data Collection / standards, ...","[Cause of Death, Data Collection / standards, ...",Trop Med Int Health,10.1111/tmi.12012
2,PMC7201103,Gender Differences in Patients With COVID-19: ...,Female,The demographic and clinical characteristics a...,"[covid-19, sars, sars-cov-2, female, gender, m...","[COVID-19 / epidemiology, COVID-19 / mortality...","[COVID-19 / epidemiology, COVID-19 / mortality...",Front Public Health,10.3389/fpubh.2020.00152


In [14]:
print(type(df.major_mesh_terms[4]))
print(type(MeSH.list_to_dict(df.major_mesh_terms[4])))
print(MeSH.list_to_dict(df.major_mesh_terms[4]))

<class 'numpy.ndarray'>
<class 'dict'>
{'therapy': {'Intracranial Arteriovenous Malformations', 'Arteriovenous Fistula'}, 'pathology': {'Cavernous Sinus', 'Carotid Artery, Internal'}, 'complications': {'Ehlers-Danlos Syndrome'}, 'methods': {'Embolization, Therapeutic', 'Endovascular Procedures'}}


In [15]:
df['dict_mesh'] = df.major_mesh_terms.apply(MeSH.list_to_dict)
df.head()

,pmcid,title,gender,case_text,keywords,major_mesh_terms,mesh_terms,journal,doi,dict_mesh
0,PMC5287946,Malignant adenohypophysis spindle cell oncocyt...,Male,A 30-year-old man came to Peking Union Medical...,None,"[Adenoma, Oxyphilic / pathology, Ki-67 Antigen...","[Adenoma, Oxyphilic / pathology, Ki-67 Antigen...",Medicine (Baltimore),10.1097/MD.0000000000005657,"{'pathology': {'Neoplasm Recurrence, Local', '..."
1,PMC3678730,Misclassification of pregnancy-related deaths ...,Female,"For each death, the DSS has systematically rec...",None,"[Cause of Death, Data Collection / standards, ...","[Cause of Death, Data Collection / standards, ...",Trop Med Int Health,10.1111/tmi.12012,"{'No Qualifier': {'Mortality', 'Maternal Morta..."
2,PMC7201103,Gender Differences in Patients With COVID-19: ...,Female,The demographic and clinical characteristics a...,"[covid-19, sars, sars-cov-2, female, gender, m...","[COVID-19 / epidemiology, COVID-19 / mortality...","[COVID-19 / epidemiology, COVID-19 / mortality...",Front Public Health,10.3389/fpubh.2020.00152,"{'epidemiology': {'COVID-19', 'Severe Acute Re..."
3,PMC4108191,Acute pulmonary embolism caused by enlarged ut...,Female,"A 42-year-old, non-smoking, virgin woman prese...","[echocardiography, leiomyoma, pulmonary emboli...","[Pulmonary Embolism / etiology, Uterine Neopla...","[Pulmonary Embolism / etiology, Uterine Neopla...",Am J Case Rep,10.12659/AJCR.890607,"{'etiology': {'Pulmonary Embolism'}, 'complica..."
4,PMC4508704,Transvenous embolization for carotid-cavernous...,Female,The patient was a 37-year-old woman with a his...,None,"[Arteriovenous Fistula / therapy, Carotid Arte...","[Arteriovenous Fistula / therapy, Carotid Arte...",Neurol Med Chir (Tokyo),10.2176/nmc.cr.2013-0007,{'therapy': {'Intracranial Arteriovenous Malfo...


In [52]:
i=4
for key in df.dict_mesh[i].keys():
    mesh_terms = df.dict_mesh[i][key]  # C'est un set
    print("\nQualificateur :", key)

    for term in mesh_terms:  # Itérer sur le set (même si 1 seul élément)
        print(term)
        print(MeSH.recherche_descripteur(racine, str(term)))


Qualificateur : therapy
Intracranial Arteriovenous Malformations
{'identifiant': 'D002538', 'nom': 'Intracranial Arteriovenous Malformations', 'numeros_mesh': ['C10.228.140.300.520', 'C10.500.190.500', 'C14.240.850.750.295', 'C14.240.850.875.500', 'C14.907.150.295', 'C14.907.253.560.400', 'C16.131.240.850.750.295', 'C16.131.240.850.875.500', 'C16.131.666.190.500']}
Arteriovenous Fistula
{'identifiant': 'D001164', 'nom': 'Arteriovenous Fistula', 'numeros_mesh': ['C14.240.850.750.147', 'C14.240.850.984.750', 'C14.907.150.125', 'C14.907.933.555', 'C16.131.240.850.750.125', 'C23.300.575.950.250']}

Qualificateur : pathology
Cavernous Sinus
{'identifiant': 'D002426', 'nom': 'Cavernous Sinus', 'numeros_mesh': ['A07.015.908.224.334']}
Carotid Artery, Internal
{'identifiant': 'D002343', 'nom': 'Carotid Artery, Internal', 'numeros_mesh': ['A07.015.114.186.200.230']}

Qualificateur : complications
Ehlers-Danlos Syndrome
{'identifiant': 'D004535', 'nom': 'Ehlers-Danlos Syndrome', 'numeros_mesh':

In [32]:
MeSH.recherche_descripteur(racine, 'Pulmonary Embolism')

{'identifiant': 'D011655',
 'nom': 'Pulmonary Embolism',
 'numeros_mesh': ['C08.381.746', 'C14.907.355.350.700']}